In [1]:
!mkdir data/ 
!wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar -O data/ILSVRC2012_img_val.tar
!tar -xf data/ILSVRC2012_img_val.tar -C data/
!wget https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
!wget https://raw.githubusercontent.com/yqihao/PTMValidations/main/imagenet_classes.txt
!wget https://raw.githubusercontent.com/yqihao/PTMValidations/main/ILSVRC2012_validation_ground_truth.txt
!rm data/ILSVRC2012_img_val.tar

--2022-05-04 00:56:33--  https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6744924160 (6.3G) [application/x-tar]
Saving to: ‘data/ILSVRC2012_img_val.tar’

data/ILSVRC2012_img 100%[===================>]   6.28G  11.8MB/s    in 9m 5s   

2022-05-04 01:05:38 (11.8 MB/s) - ‘data/ILSVRC2012_img_val.tar’ saved [6744924160/6744924160]

--2022-05-04 01:06:09--  https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24366 (24K) [text/plain]
Saving to: ‘map_clsloc.txt’

map_clsloc.txt      100%[====

In [6]:
import tensorflow as tf
import numpy as np

model = tf.keras.applications.EfficientNetB5(weights='imagenet')

mapping_file = open('map_clsloc.txt',mode='r')

validation_file = open('ILSVRC2012_validation_ground_truth.txt',mode='r')

validation_list = validation_file.readlines()
mapping_list_from = mapping_file.readlines()
mapping_list_to = []
for i in range(len(mapping_list_from)):
  mapping_list_to.append(mapping_list_from[i].split(" ")[1])
  mapping_list_from[i] = mapping_list_from[i].split(" ")[0]

for i in range(len(validation_list)):
  validation_list[i] = validation_list[i].split('\n')[0]

123478016/123465288 [==============================] - 3s 0us/step


In [9]:
correct = 0
top_5 = 0

it = 50000

for i in range(it):
  img_path = f'data/ILSVRC2012_val_{(i+1):08d}.JPEG'
  img = tf.keras.preprocessing.image.load_img(img_path, target_size=(456, 456))
  x = tf.keras.preprocessing.image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = tf.keras.applications.efficientnet.preprocess_input(x)
  preds = model.predict(x)

  id = tf.keras.applications.imagenet_utils.decode_predictions(preds, top=1)[0][0][0]
  index = mapping_list_from.index(id)

  if mapping_list_to[index] == validation_list[i]:
    correct = correct + 1
    top_5 = top_5 + 1
  else:
    for ii in range(1,5):
      id = tf.keras.applications.imagenet_utils.decode_predictions(preds, top=5)[0][ii][0]
      index = mapping_list_from.index(id)
      if mapping_list_to[index] == validation_list[i]:
        top_5 = top_5 + 1
        break

  if i!=0 and i%500==0:
    print(str(int(i/500))+'%, Top 1:' + str(correct/i) + ' Top 5:' + str(top_5/i))

accuracy = correct/it*100
top_5_accuracy = top_5/it*100
print('Top 1 Accuracy:'+str(accuracy)+'%')
print('Top 5 Accuracy:'+str(top_5_accuracy)+'%')

1%, Top 1:0.792 Top 5:0.978
2%, Top 1:0.803 Top 5:0.976
3%, Top 1:0.8226666666666667 Top 5:0.9733333333333334
4%, Top 1:0.822 Top 5:0.9685
5%, Top 1:0.8236 Top 5:0.9676
6%, Top 1:0.8243333333333334 Top 5:0.9663333333333334
7%, Top 1:0.8228571428571428 Top 5:0.9665714285714285
8%, Top 1:0.823 Top 5:0.96625
9%, Top 1:0.8244444444444444 Top 5:0.9651111111111111
10%, Top 1:0.8234 Top 5:0.9646
11%, Top 1:0.8243636363636364 Top 5:0.9636363636363636
12%, Top 1:0.8255 Top 5:0.9645
13%, Top 1:0.827076923076923 Top 5:0.9644615384615385
14%, Top 1:0.8258571428571428 Top 5:0.9647142857142857
15%, Top 1:0.8272 Top 5:0.9641333333333333
16%, Top 1:0.82775 Top 5:0.963625
17%, Top 1:0.8287058823529412 Top 5:0.964
18%, Top 1:0.8262222222222222 Top 5:0.9635555555555556
19%, Top 1:0.8274736842105264 Top 5:0.9636842105263158
20%, Top 1:0.8271 Top 5:0.9631
21%, Top 1:0.8271428571428572 Top 5:0.963047619047619
22%, Top 1:0.8251818181818181 Top 5:0.9621818181818181
23%, Top 1:0.824695652173913 Top 5:0.9623478